In [ ]:
## this is incomplete!!!

In [ ]:
## generate proteomic taxonomy plots
# uses data from analysis2016_07_28 and grouped loci filtered and normalized
# yates_normalization and min_quant = 10 in 1 sample, and min_quant = 2 in 2 samples per sample_type

In [12]:
import sys
import os
BASE = "/home/gstupp/projects/Wolan/cmoon/CM7_CM1E2d56col_unenr123_rawextract_2017/"
OUT = os.path.join(BASE, "out")
#sys.path.insert(0, "/home/gstupp/projects/metaproteomics/")
import pandas as pd
import numpy as np
import shelve
import pickle
from itertools import chain
from collections import defaultdict, Counter
from metaproteomics import utils
from tqdm import tqdm
from metaproteomics.analysis import taxonomy
from metaproteomics.analysis import DBInfo
from metaproteomics.analysis import build_loci
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
!ssh -fNL 27028:localhost:27018 gbwl
!ssh -fNL 27027:localhost:27017 gbwl

packet_write_wait: Connection to UNKNOWN port 0: Broken pipe
packet_write_wait: Connection to UNKNOWN port 0: Broken pipe


In [2]:
dbinfo = DBInfo.DBInfo('compil_mgm_localhost')
from metaproteomics.file_processing import blazmass_tools
t = taxonomy.Taxonomy(host="localhost", port=27027)
from pymongo import MongoClient
protDB = dbinfo.protDB
seqDB = dbinfo.seqDB
taxDB = MongoClient('localhost', 27027)["TaxDB_20151009"]["TaxDB_20151009"]
taxDB.find_one()

{'_id': 2,
 'exact_match': True,
 'matched_organism': 'Paenibacillus phage phiIBB_Pl23',
 'organism': 'Paenibacillus phage phiIBB_Pl23',
 'taxid': 1337877}

In [3]:
samples_keep = {'RTsep_unenr1','RTsep_unenr2','RTsep_unenr3','Ragsep_unenr1','Ragsep_unenr2','Ragsep_unenr3','WTsep_unenr1','WTsep_unenr2','WTsep_unenr3'}

In [4]:
samples = shelve.open(os.path.join(OUT,"samples.shelve"))
#sample_name = "RTsep_unenr1"
#sample = samples[sample_name]
dfs = dict()
dfs["phylum"] = dict()
dfs["family"] = dict()

In [5]:
sample_pep_quant = dict()
sample_sum = dict()
for sample_name, sample in samples.items():
    if sample_name not in samples_keep:
        continue
    sample_pep_quant[sample_name] = blazmass_tools.build_pep_quant_dict(sample.dta_select)
    sample_sum[sample_name] = sum(sample_pep_quant[sample_name].values())

nf = {k:v/np.median(list(sample_sum.values())) for k,v in sample_sum.items()}

In [6]:
sample_name, sample = next(iter(samples.items()))

In [7]:
def get_pep_tax(peptides):
    """
    for each peptide, get all taxIDs that can match it
    """
    pep_tax = dict()
    for peptide in peptides:
        p = set(x['i'] for x in seqDB.find_one(peptide)['p']) & sample.prot_ids
        x = list(taxDB.aggregate([{'$match':{'_id':{'$in':list(p)}}},{'$group':{'_id':None,'taxid':{'$addToSet':"$taxid"}}}]))
        if x:
            taxIDs=[y for y in x[0]['taxid'] if y]
            pep_tax[peptide] = taxIDs
    return pep_tax

def get_pep_lca(pep_tax):
    # for each peptide: list of taxIDs, return the LCA for that peptide
    all_taxids = set(chain(*pep_tax.values()))
    pep_lca = dict()
    for pep, taxIDs in pep_tax.items():
        pep_lca[pep] = t.LCA(taxIDs)
    return pep_lca


def get_organism_spec_pep(pep_lca, sample_name):
    pep_lcatax = dict()
    pep_lca = {k:v for k,v in pep_lca.items() if v}
    all_taxids = set(pep_lca.values())
    taxdict = {x['taxid']:x for x in t.taxonomy_coll.find({'taxid': {'$in': list(all_taxids)}})}


    organism_pep = defaultdict(set)
    pep_organism = {pep:taxdict[taxid]['scientific_name'] for pep,taxid in pep_lca.items()}
    for pep,organism in pep_organism.items():
        organism_pep[organism].add(pep)

    organism_pep_list = sorted(list(organism_pep.items()), key=lambda x:len(x[1]), reverse=True)
    return organism_pep_list

In [8]:
pep_lca = dict()
for sample_name, sample in tqdm(samples.items()):
    peptides = blazmass_tools.get_unmod_peptides(sample.dta_select)
    #peptides = list(peptides)[:100]
    pep_tax = get_pep_tax(peptides)
    pep_lca[sample_name] = get_pep_lca(pep_tax)
    """
    organism_pep_list = get_organism_spec_pep(pep_lca, sample_name)
    with open(os.path.join(OUT,"organism_specific_peptides_{}.csv".format(sample_name)), 'w') as f:
        f.writelines('\n'.join([x[0] + ";" + ",".join(x[1]) for x in organism_pep_list]))
    """

100%|██████████| 9/9 [39:21<00:00, 229.06s/it]5it/s]


In [16]:
with open("pep_lca", 'wb') as f:
    pickle.dump(pep_lca, f)

In [9]:
for sample_name, plca in pep_lca.items():
    for rank in ["phylum", "family"]:
        organism_pep = defaultdict(set)
        for pep, lca in tqdm(plca.items()):
            family = t.get_rank(lca, rank, "taxid")
            if family:
                organism_pep[family].add(pep)

        organism_quant = {k:sum([sample_pep_quant[sample_name][pep] for pep in peps]) for k,peps in organism_pep.items()}

        df = pd.DataFrame(organism_quant, index = ['count']).T
        df['organism_name'] = [t.taxid_to_taxonomy(int(x))['scientific_name'] for x in df.index]
        df=df.sort_values("count",ascending=False)

        df.to_csv(os.path.join(OUT,"{}_count_{}.csv".format(rank, sample_name)))
        dfs[rank][sample_name] = df

100%|██████████| 6320/6320 [04:44<00:00, 22.21it/s]


In [17]:
for rank in ["phylum", "family"]:
    for sample_name, df in dfs[rank].items():
        df['taxid'] = df.index
        df['sample'] = sample_name
    all_df = pd.concat(list(dfs[rank].values()))
    all_df.to_csv(os.path.join(OUT,"{}_count.csv".format(rank)))

In [20]:
for rank in ["phylum", "family"]:
    all_df = pd.concat(list(dfs[rank].values()))
    df_pivot = all_df.pivot_table(values="count", index="sample", columns="organism_name").dropna(1, thresh=3)
    df_pivot.to_csv(os.path.join(OUT,"{}_count_pivot.csv".format(rank)))